# Regrouping JSON with jq

As a starting point, we have a JSON file where 
AWS accounts are enumerated, and each account has
a list of regions defined.

In [1]:
<input.json jq

[
  {
    "account": "111111111111",
    "regions": [
      "us-east-1",
      "eu-central-1"
    ]
  },
  {
    "account": "222222222222",
    "regions": [
      "us-west-2",
      "us-east-1"
    ]
  }
]


Our desired output is to have a JSON file where
regions are enumerated, and each region has a list
of accounts defined.

In [2]:
<desired.json jq

[
  {
    "accounts": [
      "111111111111",
      "222222222222"
    ],
    "region": "us-east-1"
  },
  {
    "accounts": [
      "222222222222"
    ],
    "region": "us-west-2"
  }
]


We will be capturing intermediate state in the `out` directory.

In [3]:
mkdir -p out

## Ungrouping

In it's initial form the input is grouped by accounts.

We want to ungroup the data, and have a record for each account and region pair.

First, for each element in the list, we take the `regions` list and start iterating over it (`.regions[]`), capturing the current value in the `$region`.

We then construct an object for each `$region` with two fields: the .account field in the root object, and the actual $region in the iteration.

If we were to do that in python it would look like this:

```python
result = []
with open('input.json') as file:
    items = json.load(file)
    for item in items:
        for region in item['regions']:
            result.push({'account': item['account'], 'region': region})
print(json.dumps(result, indent=4))
```

With jq we can do that in a bit more concise way.

In [4]:
<input.json jq 'map(.regions[] as $region | {account, $region})' \
| tee out/step-1.json | jq

[
  {
    "account": "111111111111",
    "region": "us-east-1"
  },
  {
    "account": "111111111111",
    "region": "eu-central-1"
  },
  {
    "account": "222222222222",
    "region": "us-west-2"
  },
  {
    "account": "222222222222",
    "region": "us-east-1"
  }
]


# Regrouping
Now that the data is ungrouped, we can move on to grouping it by region.

In [5]:
<out/step-1.json jq 'group_by(.region)' \
| tee out/step-2.json | jq

[
  [
    {
      "account": "111111111111",
      "region": "eu-central-1"
    }
  ],
  [
    {
      "account": "111111111111",
      "region": "us-east-1"
    },
    {
      "account": "222222222222",
      "region": "us-east-1"
    }
  ],
  [
    {
      "account": "222222222222",
      "region": "us-west-2"
    }
  ]
]


Next step is to merge records into a single one.

We will create an `accounts` field, where all account numbers for a region are 
enumerated in a list.

The region of a group we can take from the first element in that group, since all elements
have the same region anyways.

In [6]:
<out/step-2.json jq 'map({region: .[0].region, accounts: map(.account)})' \
| tee out/step-3.json | jq

[
  {
    "region": "eu-central-1",
    "accounts": [
      "111111111111"
    ]
  },
  {
    "region": "us-east-1",
    "accounts": [
      "111111111111",
      "222222222222"
    ]
  },
  {
    "region": "us-west-2",
    "accounts": [
      "222222222222"
    ]
  }
]


## Summary

All of the above can be put into a single jq expression

In [7]:
<input.json jq '
    map(
        .regions[] as $region
        | {
            account,
            $region
        })
    | group_by(.region)
    | map({
        accounts: map(.account), 
        region: .[0].region
    })'

[
  {
    "accounts": [
      "111111111111"
    ],
    "region": "eu-central-1"
  },
  {
    "accounts": [
      "111111111111",
      "222222222222"
    ],
    "region": "us-east-1"
  },
  {
    "accounts": [
      "222222222222"
    ],
    "region": "us-west-2"
  }
]
